In [1]:
import sys
sys.path.append("../src")

import ollama
from torch import load

import gradio as gr
import time
from functions import *

In [2]:
# pull model
ollama.pull('llama3.2-vision')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [5]:
# load article contents
text_content_list = load_from_json('../data/processed_files/JSON/text_content.json')
image_content_list = load_from_json('../data/processed_files/JSON/image_content.json')

# load embeddings
text_embeddings = load('../data/processed_files/embeddings/text_embeddings.pt', weights_only=True)
image_embeddings = load('../data/processed_files/embeddings/image_embeddings.pt', weights_only=True)

In [6]:
# Function to interact with the Ollama model
def stream_chat(message, history):
    """
    Streams the response from the Ollama model and sends it to the Gradio UI.
    
    Args:
        message (str): The user input message.
        history (list): A list of previous conversation messages.
        
    Yields:
        str: The chatbot's response chunk by chunk.
    """

    # context retrieval
    text_results, image_results = context_retrieval(message["text"], text_embeddings, image_embeddings, text_content_list, image_content_list)

    # construct prompt
    prompt = construct_prompt(message["text"], text_results, image_results)
    
    # Append the user message to the conversation history
    history.append({"role": "user", "content": prompt, "images": [image["image_path"] for image in image_results]})
    
    # Initialize streaming from Ollama
    stream = ollama.chat(
        model='llama3.2-vision',
        messages=history,  # Full chat history including the current user message
        stream=True,
    )
    
    response_text = ""
    for chunk in stream:
        content = chunk['message']['content']
        response_text += content
        yield response_text  # Send the response incrementally to the UI

    # Append the assistant's full response to the history
    history.append({"role": "assistant", "content": response_text})

In [7]:
# Create a Gradio ChatInterface
gr.ChatInterface(
    fn=stream_chat,  # The function handling the chat
    type="messages",  # Using "messages" to enable chat-style conversation
    examples=[{"text": "What does this document talk about?"}, 
              {"text": "What do the images in this document show?"},],  # Example inputs
    multimodal=True,
).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Traceback (most recent call last):
  File "C:\Users\leopa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\leopa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\leopa\AppData\Local\Package